# Пособие по регулярным выражениям
## Предисловие
#### Сплошь и рядом программисты традиционного (Windows) направления не владеют регулярными выражениями. От слова совсем. Те, кто программирует на SQL, как правило, владеют шаблонами поиска и функцией PATINDEX. Это уже ближе к регулярным выражениям, но пока еще все-таки очень далеко. Программисты Web направления, наоборот, чаще всего владеют регулярными выражениями, но, надеюсь, и они почерпнут что-то полезное для себя из этой статьи.
#### Когда я сам начинал изучать регулярные выражения, было очень не легко справиться с большим количеством информации, которая, к тому же, подается в совершенно неправильном порядке, или просто хаотично (Microsoft-style). Если начинать изучение регулярных выражений с кванторов, особенно жадных и ленивых кванторов (greedy and lazy quantors), то порог овладения этой технологией повышается кратно. Моя же задача - понизить этот порог, сделать так, чтобы выражения "import re" в  Python, либо  "using System.Text.RegularExpression" в С# встречались как можно чаще.
#### В этой статье я буду использовать Python и re. У C# некоторые моменты имеют отличия, но об этом - в конце статьи.
#### Итак, приступим...

In [2]:
import re

## Глава 1 - Функции
#### Что вообще можно сделать при помощи re? Четыре возможных действия - 1) найти подстроку в строке; 2) найти все подстоки в строке; 3) разделение строки на подстроки; 4) заменить одну подстроку на другую в строке. Просто? Да, совершенно. Сложность лишь в том, что подстроки во всех этих функциях задаются шаблонами. А шаблоны могут быть очень-очень сложными.
#### 1) _match_ и _search_ - ищут шаблон подстроки в строке. В Python _match_ ищет с начала строки, предпочтительнее использовать _search_, в C# функции _search_ нет, используйте _match_.
#### 2) _findall_ (в C# - _matches_) - находит все вхождения шаблона подстроки в строку.
#### 3) _sub_ (в C# - _replace_) - выполняет замену подстроки на другую подстроку, при этом допускается использование найденных шаблонов из этого же регулярного выражения.
#### 4) _split_-разделяет строку на массив строк.
#### Простые примеры приведены ниже (в Python string и unicode - разные типы, пока работаем со  string, значит буквы пока будут латинские)

In [16]:
strings = ['The first string', 'The second string', 'My phone number is 777-778-92345', 'A, B: C# D']

#search & match
print(u'Ищем слово "first" в строке "%s"' % strings[0])
search = re.search('first', strings[0])
match = re.match('first', strings[0])
if search is not None:
    print('Search - Found %s at position %d' % (search.group(0), search.start()))
else:
    print('Search - Pattern "first" not found')
if match is not None:
    print('Match - Found %s at position %d' % (search.group(0), search.start()))
else:
    print('Match - Pattern "first" not found') #pattern "first" is not first word of phrase

#findall
print(u'Ищем слово "second" в строке "%s"' % strings[1])
print(re.findall('second', strings[1]))
print(u'Ищем слова, которые начинаются с s в строке "%s"' % strings[1])
print(re.findall(r'\bs\w+', strings[1])) #see explanation below

#sub
print(u'Выполняем замену символов в строке "%s"' % strings[2])
print(re.sub(r'\d+', 'XXX', strings[2]))

#split
print(u'Разделяем строку "%s" на элементы' % strings[3])
print(re.split(r'\W+', strings[3]))

Ищем слово "first" в строке "The first string"
Search - Found first at position 4
Match - Pattern "first" not found
Ищем слово "second" в строке "The second string"
['second']
Ищем слова, которые начинаются с s в строке "The second string"
['second', 'string']
Выполняем замену символов в строке "My phone number is 777-778-92345"
My phone number is XXX-XXX-XXX
Разделяем строку "A, B: C# D" на элементы
['A', 'B', 'C', 'D']


## Шаблоны поиска
#### Шаблоны поиска - это смешные символы  \b, \w, \W, которые я употребил в предыдущих примерах (специальный символ "+" - это квантор, он отвечает за количество, о нем - ниже). В примере выше - \b означает символ начала или конца слова, \w - буква или цифра, \W - ни буква, ни цифра. Наиболее употребительные:
#### . - любой символ; если нужно найти именно ".", то нужно писать "\."
#### ^ - начало строки
#### "_$_" - конец строки
#### \s - символ whitespace, например, пробел или табуляция, \S - не whitespace
#### \d - цифра 0-9, \D - не цифра
#### про \w и \W упомянуто выше. Вообще, можно с помощью [] создать шаблон любого набора символов. Например, 
#### "[1-5a-zГ-У]" будет соответствововать символам от 1 до 5, либо от a до z, либо от Г до У включительно.
#### "[^1-5a-zГ-У]" будет соответствовать всем символам НЕ из этого набора
#### также существует общее правило - есть вы ищете символ, который для  re является специальным (например, ^), просто ставьте перед ним обратный слэш "\"

## Флаги
#### На поведение  шаблонов поиска существенно влияют так называемые флаги. Указываются они как дополнительная опция поиска, либо при компиляции регулярного выражения. Вот список этих флагов и их влияние на результат:
#### re.DEBUG - включение отладочной информации при компиляции re
#### re.IGNORECASE (re.I) - для регистронезависимого поиска, но имеет значение флаг re.U при поиске и сравнении русских символов
#### re.LOCALE (re.L) - для применения локальных настроек 
#### re.UNICODE (re.U) - включает "знание" русского алфавита, без этой настройки \w не узнает русские буквы и re.I некорректно обрабатывает регистр
#### re.MULTILINE (re.M) - важно для обработки многострочных текстов, влияет на поведение '^'; с этим флагом '^' будет соответствовать началу каждой новой строки
#### re.DOTALL (re.S) - без включения этого флага '.' не соответствует символу перевода строки/возврата каретки
#### re.VREBOSE re.X) - позволяет писать шаблоны в стиле user-friendly
#### При применении нескольких флагов, они перечисляются через |, например flags=re.U|re.I|re.M|re.S

In [23]:
#Some samples with flags
print(re.findall(ur'\w', u'Это строка по русски')) #found nothing
print(re.findall(ur'\w', u'Это строка по русски', flags=re.U)) #found all except whitespaces
pat = re.compile(ur"""\d+ #This is integer part of number
                    \. #Decimal point
                    \d+ #Fractional part""", flags=re.X)
print(re.findall(pat, '3.14 is pi value'))

[]
[u'\u042d', u'\u0442', u'\u043e', u'\u0441', u'\u0442', u'\u0440', u'\u043e', u'\u043a', u'\u0430', u'\u043f', u'\u043e', u'\u0440', u'\u0443', u'\u0441', u'\u0441', u'\u043a', u'\u0438']
['3.14']


## Кванторы
#### Кванторы - это специальные символы, которые управляют количеством символов по шаблону. Например, символ + после шаблона говорит о том, что ищем одно или больше совпадений, а символ *  - 0 или больше. Аналогично:
#### ? - 0 или 1 раз
#### {m,n} - не меньше m, не больше n (обратите внимание - пробелы в скобках недопустимы)
#### {m,} - m или больше
#### {,n} - n или меньше
#### {m} - точно m раз

In [30]:
string = u'Я  просто   не   умею правильно   пользоваться    пробелом'
pat = re.compile(ur'\s+', flags=re.U|re.I)
print(re.sub(pat, ' ', string))
pat = re.compile(ur'\w?р\w*', flags=re.U|re.I)
print(', '.join(re.findall(pat, string)))

Я просто не умею правильно пользоваться пробелом
просто, правильно, пробелом


## Группы
#### Группировка - это способность регулярных выражений выделить (вернуть) только часть шаблона. Например, необходимо в списке фамилий Иванов, Петров, Сидоров, Соловьев выделить корень фамилии, но только если она кончается на -ов. Можно эту задачу решить в два шага - найти фамилии, которые кончаются на -ов, а затем убрать окончание -ов, а можно вот так:

In [32]:
string = u'Иванов, Петров, Сидоров, Соловьев'
pat = re.compile(ur'(\w+)ов\b', flags=re.U|re.I)
print(', '.join(re.findall(pat, string)))

Иван, Петр, Сидор


#### Группа - часть искомого выражения "\w+ов\b", которое расшифровывается как "любое число букв или цифр, не меньшее одного, затем буквы 'ов', затем конец слова", но вернуть нам нужно только первую часть, мы выделяем эту часть круглыми скобками, это и будет группа.
#### Группы - весьма мощный инструмент регулярных выражений. Их может быть несколько в одном выражении, они могут быть именованными. Если групп несколько, то работают с ними так:

In [39]:
m = re.search(ur'(\d+)\.(\d+)', '3.14 is pi value')
print('full match is %s' % m.group(0)) #ignoring groups
print('First group is %s' % m.group(1))
print('Second group is %s' % m.group(2))
print(re.sub(ur'(\d+)\.(\d+)', '\g<2>.\g<1>', '3.14 is pi value')) #change places of groups

full match is 3.14
First group is 3
Second group is 14
14.3 is pi value


#### Круглые скобки применяются не только для групп, но полный обзор возможностей сильно усложнит материал. Достаточно знать, что с помощью круглых скобок и специальных символов можно осуществлять поиск внутри скобок поиск в обратном направлении и много чего еще, смотрите документацию. Приведу здесь всего лишь один полезный пример - комбинация (?:"regex") - игнорирует группу (здесь "regex" какое-то выражение). Практический пример шаблон (Г?ОСТ|ТУ) ищет в строке (Г)ОСТ или ТУ, но как теперь выделить цифры после?

In [41]:
strings = [u'Круг h8 - 3.3 ГОСТ 14955-77 / У10А - Б - НГ ГОСТ 1435-99',
          u'Винт В . М 18 х 1.5 - 8g х 65 . 3М ТУ 17474-80']
pat = re.compile(ur'(Г?ОСТ|ТУ)\s*\S+')
for s in strings:
    print('\n'.join(re.findall(pat, s))) #Bad result - didn't find digits due to false group
    
pat = re.compile(ur'(?:Г?ОСТ|ТУ)\s*\S+')
for s in strings:
    print('\n'.join(re.findall(pat, s))) #Solved

ГОСТ
ГОСТ
ТУ
ГОСТ 14955-77
ГОСТ 1435-99
ТУ 17474-80


## Жадные и ленвые кванторы
#### Последнее, о чем я хотел бы упомянуть, это о понятиях жадный (greedy) и ленивый (lazy), относится это к кванторам. Объявленные выше кванторы возвращают наибольшее возможное количество символов, например:

In [42]:
print(re.findall(ur'\d+', '111-2222-33333-4444444'))

['111', '2222', '33333', '4444444']


#### если же поставить "?" после "+", то мы получим ленивый квантор, который возвращает наименьшее возможное значение символов:

In [43]:
print(re.findall(ur'\d+?', '111-2222-33333-4444444'))

['1', '1', '1', '2', '2', '2', '2', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '4', '4']


#### Существенная разница, не так ли? Еще один пример - мы хотим заменить цифру 3 в примере выше - Круг h8 - 3.3 ГОСТ 14955-77 / У10А - Б - НГ ГОСТ 1435-99, но только первую цифру, которая не соседствует с другими цифрами. Элегантно решает эту задачу пара из ленивого и жадного кванторов:

In [5]:
string = 'Круг h8 - 3.3 ГОСТ 14955-77 / У10А - Б - НГ ГОСТ 1435-99'
pat = re.compile(ur'(.*?\D)(3)(\D.*)', flags=re.U|re.I)
print(re.sub(pat, r'\g<1>Ж\g<3>', string))

Круг h8 - Ж.3 ГОСТ 14955-77 / У10А - Б - НГ ГОСТ 1435-99


#### Пояснения - первая группа (.*******?\D) содержит ленивый квантор (некоторое количество символов перед не-цифрой), возвращающий наименьшее возможное число символов, а в третьей группе (\D.***) - жадный квантор, возвращающий наибольшее возможное число символов.

## Особенности использования в CSharp
#### Главное отличие - в измененном наборе опций поиска, то есть другие возможные значения переменной flags. В .NET все строки уже Unicode, поэтому там нет флага re.U, зато есть другие, за справками - https://docs.microsoft.com/ru-ru/dotnet/api/system.text.regularexpressions.regexoptions?view=netframework-4.7.2